In [1]:
import numpy as np
import pandas as pd

def generate_recommendations(enrolled_course_ids, threshold, include):
    # Load data
    if include==0:
        course_df = pd.read_csv('C:/Users/91798/E-Learning Recommendation System/filtered_courses.csv')
    else:
        course_df = pd.read_csv('C:/Users/91798/E-Learning Recommendation System/courses.csv')
        
    bows_df = pd.read_csv('C:/Users/91798/E-Learning Recommendation System/bows_df.csv')
    sim_matrix = pd.read_csv('C:/Users/91798/E-Learning Recommendation System/sim.csv').to_numpy()

    # Create index-id dictionaries
    grouped_df = bows_df.groupby(['doc_index', 'doc_id']).max().reset_index(drop=False)
    idx_id_dict = grouped_df[['doc_id']].to_dict()['doc_id']
    id_idx_dict = {v: k for k, v in idx_id_dict.items()}

    # Extract selected and unselected courses
    all_courses = set(course_df['COURSE_ID'])
    unselected_course_ids = all_courses.difference(enrolled_course_ids)

    # Generate recommendations
    res = {}
    for enrolled_course in enrolled_course_ids:
        for unselect_course in unselected_course_ids:
            if enrolled_course in id_idx_dict and unselect_course in id_idx_dict:
                idx1 = id_idx_dict[enrolled_course]
                idx2 = id_idx_dict[unselect_course]
                sim = sim_matrix[idx1][idx2]
                if sim > threshold:
                    if unselect_course not in res:
                        res[unselect_course] = sim
                    else:
                        if sim >= res[unselect_course]:
                            res[unselect_course] = sim
    sorted_recommendations = sorted(res.items(), key=lambda item: item[1], reverse=True)
    return sorted_recommendations

In [2]:
# Example usage:
enrolled_course_ids = ['ML0201EN', 'ML0122EN', 'GPXX0ZG0EN', 'RP0105EN', 'excourse54']
threshold = 0.5
recommendations = generate_recommendations(enrolled_course_ids, threshold, 0)
print(recommendations)

[('excourse40', 1.0), ('ML0122ENv3', 0.6815739441431503), ('DA0151EN', 0.5737851219446795), ('excourse69', 0.5514744732509853)]


In [4]:
enrolled_course_ids = ['ML0101EN', 'excourse46', 'excourse47']
threshold = 0.5
recommendations = generate_recommendations(enrolled_course_ids, threshold, 1)
print(recommendations)

[('ML0151EN', 0.6892532539898828), ('ML0101ENv3', 0.6347547807096177), ('excourse60', 0.6307666542363487), ('excourse69', 0.5928906206690311), ('ML0109EN', 0.5899114727287609), ('excourse61', 0.555977229601518), ('excourse51', 0.5237275549955195), ('BENTEST4', 0.5215413911459146), ('excourse52', 0.5028927502928755), ('excourse48', 0.5011224621465926), ('ML0115EN', 0.5005564831995)]


In [7]:
ratings = pd.read_csv('C:/Users/91798/E-Learning Recommendation System/ratings.csv')
rating = ratings['item'].unique()
cnt = []
for item in recommendations:
    if item[0] not in rating:
        cnt.append(item[0])
cnt

['excourse60', 'excourse61', 'excourse51', 'excourse48']